In [1]:
import ast
import os
import re

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from _import_from_src import search_log_files
from _import_from_src import LogParser
from _import_from_src import LOG_DIR

target = os.path.join(LOG_DIR, 'E2')

color_dict = {
    'GP': px.colors.qualitative.Plotly[0],
    'TP': px.colors.qualitative.Plotly[1],
    'AGT': px.colors.qualitative.Plotly[2],
}

In [57]:
class LogParser:
    def __init__(self, file_path):
        self.file_path = file_path
        self.settings = {}
        self.initial_data = {"X_initial": [], "y_initial": []}
        self.bo_data = {"X_new": [], "y_new": [], "Beta": [], "Iteration": []}
        self.objective = None

    def _combine_log_entries(self):
        with open(self.file_path, "r") as file:  # 修正: self.log_file → self.file_path
            lines = file.readlines()

        timestamp_pattern = r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3} - "

        combined_lines = []
        current_entry = ""

        for line in lines:
            if re.match(timestamp_pattern, line):
                if current_entry:
                    combined_lines.append(current_entry.strip())
                current_entry = line.strip()
            else:
                current_entry += " " + line.strip()

        if current_entry:
            combined_lines.append(current_entry.strip())

        return combined_lines

    def _parse_settings(self, line):
        settings_str = line.split("settings:")[1].strip()
        try:
            # Parse the settings safely
            self.settings = self._safe_parse_settings(settings_str)
            
            # More flexible regex to capture the class name of the objective_function
            obj_func_match = re.search(r"objective_function':\s*<.*?\.([A-Za-z0-9_]+)\s+object\s+at", settings_str)
            
            if obj_func_match:
                objective_function_name = obj_func_match.group(1)
                print(f"Objective Function: {objective_function_name}")
            else:
                print("Objective Function not found in settings.")
                
        except SyntaxError as e:
            print(f"Failed to parse settings: {e}")
            self.settings = settings_str

    def _safe_parse_settings(self, settings_str):
        # array と <function>, <class> の部分を文字列に置換
        settings_str = re.sub(r"<function\s+\w+\s+at\s+0x[0-9a-fA-F]+>", "'<function>'", settings_str)
        settings_str = re.sub(r"<class\s+'\w+\.\w+'>", "'<class>'", settings_str)
        settings_str = re.sub(r"<\w+\.\w+\s+object\s+at\s+0x[0-9a-fA-F]+>", "'<object>'", settings_str)
        
        # 辞書を文字列から生成（evalは使わず安全なast.literal_evalを使う）
        settings_dict = ast.literal_eval(settings_str)
        
        return settings_dict

    def parse_log_file(self):
        combined_lines = self._combine_log_entries()
        current_data = {}
        
        for line in combined_lines:
            # Parse settings
            if "Start BO with settings:" in line:
                settings_str = re.search(r"Start BO with settings: (.*)", line).group(1)

                try:
                    self._parse_settings(line)
                except Exception as e:
                    print(f"Failed to parse settings: {e}")
                    self.settings = settings_str
            
            # Parse X_initial and y_initial
            elif "X initial:" in line:
                x_initial_str = re.search(r"X initial: (\[.*?\])", line).group(1)
                self.initial_data["X_initial"].append(self._extract_float_list(x_initial_str))
            elif "y initial:" in line:
                y_initial_str = re.search(r"y initial: (\[.*?\])", line).group(1)
                self.initial_data["y_initial"].append(self._extract_float_list(y_initial_str))

            # Parse Beta, Iteration, X_new, and y_new
            elif "Beta:" in line:
                current_data["Beta"] = float(re.search(r"Beta: ([-+]?\d*\.\d+|\d+)", line).group(1))
            elif "Iteration:" in line:
                current_data["Iteration"] = int(re.search(r"Iteration: (\d+) /", line).group(1))
            elif "X new:" in line:
                current_data["X_new"] = self._extract_float_list(re.search(r"X new: (\[\[.*?\]\])", line).group(1))
            elif "y new:" in line:
                current_data["y_new"] = self._extract_float_list(re.search(r"y new: (\[\[.*?\]\])", line).group(1))

                # Check if all required fields are in `current_data`
                if all(key in current_data for key in ["Beta", "Iteration", "X_new", "y_new"]):
                    # Append the current data to `self.bo_data`
                    self.bo_data["Beta"].append(current_data["Beta"])
                    self.bo_data["Iteration"].append(current_data["Iteration"])
                    self.bo_data["X_new"].append(current_data["X_new"])
                    self.bo_data["y_new"].append(current_data["y_new"])

                    # Clear current_data for the next entry
                    current_data.clear()
        
        # pandas DataFrame に変換
        self.initial_data = pd.DataFrame(self.initial_data)
        self.bo_data = pd.DataFrame(self.bo_data)

        self.initial_data["X_initial"] = self.initial_data["X_initial"].apply(np.array)
        self.initial_data["y_initial"] = self.initial_data["y_initial"].apply(np.array)
        self.initial_data["y_initial"] = self.initial_data["y_initial"].apply(float)

        self.bo_data["X_new"] = self.bo_data["X_new"].apply(np.array)
        self.bo_data["y_new"] = self.bo_data["y_new"].apply(np.array)
        self.bo_data["y_new"] = self.bo_data["y_new"].apply(float)
        

    def _extract_float_list(self, array_str):
        # Updated regex pattern to capture numbers in scientific notation as well
        number_pattern = r"[-+]?\d*\.\d+(?:[eE][-+]?\d+)?|\d+"
        numbers = re.findall(number_pattern, array_str)
        return [float(num) for num in numbers]
    
    def create_combined_dataframe(self):
        beta_init = self.bo_data['Beta'].iloc[0]

        df_init = self.initial_data.copy()
        df_init['X_new'] = df_init['X_initial']
        df_init['y_new'] = df_init['y_initial']
        df_init.drop(columns=['X_initial', 'y_initial'], inplace=True)
        df_init['y_best'] = df_init['y_new'].min()
        df_init['Beta'] = beta_init
        df_init['Beta:n'] = beta_init / len(df_init)
        df_init['Iteration'] = 0

        df_main = self.bo_data.copy()
        df_main['y_best'] = df_main['y_new'].cummin()
        df_main['y_best'] = np.minimum(df_main['y_best'], df_init['y_best'].iloc[0])
        df_main['Beta:n'] = df_main['Beta'] / (len(df_init) + df_main.index)

        df = pd.concat([df_init, df_main])
        return df 
    
# Create a history plot for bo_data using Plotly
def history_plot(bo_data):
    fig = go.Figure()

    # Plot y_new as blue dots
    fig.add_trace(go.Scatter(
        x=bo_data['Iteration'],
        y=bo_data['y_new'],
        mode='markers',
        name='y_new',
        marker=dict(color='blue')
    ))

    # Plot y_best as a blue line
    fig.add_trace(go.Scatter(
        x=bo_data['Iteration'],
        y=bo_data['y_best'],
        mode='lines',
        name='y_best',
        line=dict(color='blue')
    ))

    # Highlight points where Beta:n >= 1 with orange dots
    high_beta_mask = bo_data['Beta:n'] >= 1
    fig.add_trace(go.Scatter(
        x=bo_data['Iteration'][high_beta_mask],
        y=bo_data['y_new'][high_beta_mask],
        mode='markers',
        name='y_new (Beta:n >= 1)',
        marker=dict(color='orange')
    ))

    # Customize the layout
    fig.update_layout(
        title="Bayesian Optimization History",
        xaxis_title="Iteration",
        yaxis_title="Function Value",
        legend_title="Metrics"
    )

    fig.show()


def process_dataframes(dataframes):
    # Find the minimum number of rows across all DataFrames
    min_rows = min(df.shape[0] for df in dataframes)
    max_rows = max(df.shape[0] for df in dataframes)

    # Truncate each DataFrame to the minimum number of rows
    truncated_dfs = [df[['y_best', 'Iteration', 'Beta']].head(min_rows) for df in dataframes]

    # Concatenate the truncated DataFrames along the row axis
    concatenated_df = pd.concat(truncated_dfs)

    # Group by the iteration and calculate the mean and std for y_best
    result_df = concatenated_df.groupby('Iteration').agg(
        y_best_mean=('y_best', 'mean'),
        y_best_std=('y_best', 'std'),
        beta_mean=('Beta', 'mean'),
        beta_std=('Beta', 'std')
    ).reset_index()

    # Print the minimum and maximum row counts
    print(f"Minimum number of rows: {min_rows}")
    print(f"Maximum number of rows: {max_rows}")
    
    return result_df

def add_history_plot_with_error_bars(fig, model_name, bo_data):
    # Plot y_best without error bars (for mean line and markers)
    fig.add_trace(go.Scatter(
        x=bo_data['Iteration'],
        y=bo_data['y_best_mean'],
        mode='lines+markers',
        name=f"{model_name} (Mean)",
        line=dict(color=color_dict[model_name])
    ))

    # Plot the same y_best with error bars and reduced opacity for the second trace
    fig.add_trace(go.Scatter(
        x=bo_data['Iteration'],
        y=bo_data['y_best_mean'],
        mode='lines+markers',
        name=f"{model_name} (Error Bars)",
        line=dict(color=color_dict[model_name]),
        opacity=0.3,  # Reduce opacity for the second trace (error bars)
        error_y=dict(
            type='data',  # Error type is 'data', which means we provide error bars as data
            array=bo_data['y_best_std'],  # Use standard deviation as error bars
            visible=True  # Make sure error bars are visible
        ),
        showlegend=False  # Hide this trace from the legend
    ))

# Function to create and store result_df for each model
def create_result_df_dict(target, models, keywards):
    result_df_dict = {}
    
    # Loop through each model to process files and create result_df
    for model_name in models:
        # Update keywards with the current model name
        updated_keywards = [model_name] + keywards  # Replace the first item with model_name
        
        # Search for log files
        filename_list = search_log_files(target, updated_keywards)
        filepath_list = [os.path.join(target, filename) for filename in filename_list]
        
        # Initialize an empty list to hold dataframes
        dataframes = []
        
        # Parse each log file and create dataframes
        for filepath in filepath_list:
            parser = LogParser(filepath)
            parser.parse_log_file()
            df = parser.create_combined_dataframe()
            dataframes.append(df)
        
        # Process the dataframes and store the result in result_df_dict
        result_df = process_dataframes(dataframes)
        result_df_dict[model_name] = result_df  # Store the result in the dictionary with the model name as key
    
    return result_df_dict

In [16]:
keywards = ["AGT", 'Sinusoidal', 'EI']
filename_list = search_log_files(target, keywards)
filename_list

['2024-09-21_22-15-14_roposed_bo_SinusoidalSynthetic_AGT_EI_seed0.log',
 '2024-09-21_22-15-14_roposed_bo_SinusoidalSynthetic_AGT_EI_seed1.log',
 '2024-09-21_22-15-14_roposed_bo_SinusoidalSynthetic_AGT_EI_seed2.log',
 '2024-09-21_22-15-14_roposed_bo_SinusoidalSynthetic_AGT_EI_seed3.log',
 '2024-09-21_22-15-14_roposed_bo_SinusoidalSynthetic_AGT_EI_seed4.log']

In [22]:
filename = '2024-09-21_22-15-14_roposed_bo_SinusoidalSynthetic_AGT_EI_seed0.log'
file_path = os.path.join(target, filename)

parser = LogParser(file_path)
parser.parse_log_file()
df = parser.create_combined_dataframe()
df

Failed to parse settings: invalid syntax (<unknown>, line 1)


,X_new,y_new,y_best,Beta,Beta:n,Iteration
0,[6.97321758],21.465557,-21.104920,2.168405,0.542101,0
1,[8.8004753],-18.257951,-21.104920,2.168405,0.542101,0
2,[7.89199717],-8.354414,-21.104920,2.168405,0.542101,0
3,[6.00610279],-21.104920,-21.104920,2.168405,0.542101,0
0,[5.0],15.382359,-21.104920,2.168405,0.542101,1
1,[10.0],-6.801930,-21.104920,2.909167,0.581833,2
2,[9.3801155],69.193214,-21.104920,3.665157,0.610859,3
3,[5.9179335],-16.470800,-21.104920,4.449562,0.635652,4
4,[5.710989],-3.512074,-21.104920,5.288769,0.661096,5
5,[5.963643],-18.985937,-21.104920,6.313912,0.701546,6


In [23]:
# Apply the function to plot the bo_data
history_plot(df)

In [49]:
target = os.path.join(LOG_DIR, 'E2')
keywards = ["AGT", 'Sinusoidal', 'EI']
filename_list = search_log_files(target, keywards)
print(filename_list)
filepath_list = [os.path.join(target, filename) for filename in filename_list]

dataframes = []
for filepath in filepath_list:
    parser = LogParser(filepath)
    parser.parse_log_file()
    df = parser.create_combined_dataframe()
    dataframes.append(df)

result_df = process_dataframes(dataframes)
result_df

['2024-09-21_22-15-14_roposed_bo_SinusoidalSynthetic_AGT_EI_seed0.log', '2024-09-21_22-15-14_roposed_bo_SinusoidalSynthetic_AGT_EI_seed1.log', '2024-09-21_22-15-14_roposed_bo_SinusoidalSynthetic_AGT_EI_seed2.log', '2024-09-21_22-15-14_roposed_bo_SinusoidalSynthetic_AGT_EI_seed3.log', '2024-09-21_22-15-14_roposed_bo_SinusoidalSynthetic_AGT_EI_seed4.log']
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 50
Maximum number of rows: 51


,Iteration,y_best_mean,y_best_std,beta_mean,beta_std
0,0,-17.146889,8.036184,2.166456,0.016498
1,1,-17.798055,7.527830,2.166456,0.017978
2,2,-17.798055,7.527830,2.939141,0.046523
3,3,-17.798055,7.527830,3.817778,0.132035
4,4,-17.798055,7.527830,4.735809,0.198703
5,5,-18.527262,7.582573,5.788246,0.439637
6,6,-19.910740,7.838349,6.614577,0.329828
7,7,-24.984307,2.431712,7.585626,0.698519
8,8,-25.585718,2.114341,8.491489,1.300994
9,9,-26.112342,1.555310,9.969388,2.538477


In [33]:
target = os.path.join(LOG_DIR, 'E2')
keywards = ["GP", 'Sinusoidal', 'EI']
filename_list = search_log_files(target, keywards)
print(filename_list)
filepath_list = [os.path.join(target, filename) for filename in filename_list]

dataframes = []
for filepath in filepath_list:
    parser = LogParser(filepath)
    parser.parse_log_file()
    df = parser.create_combined_dataframe()
    dataframes.append(df)

result_df = process_dataframes(dataframes)
result_df

['2024-09-21_22-18-31_vanilla_bo_SinusoidalSynthetic_GP_EI_seed0.log', '2024-09-21_22-18-31_vanilla_bo_SinusoidalSynthetic_GP_EI_seed1.log', '2024-09-21_22-18-31_vanilla_bo_SinusoidalSynthetic_GP_EI_seed2.log', '2024-09-21_22-18-31_vanilla_bo_SinusoidalSynthetic_GP_EI_seed3.log', '2024-09-21_22-18-31_vanilla_bo_SinusoidalSynthetic_GP_EI_seed4.log']
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 54
Maximum number of rows: 57


,Iteration,y_best_mean,y_best_std,beta_best_mean,beta_best_std
0,0,-17.838015,10.742664,1.513816,0.097179
1,1,-17.838015,11.706547,1.513816,0.105898
2,2,-20.942302,11.864001,2.009857,0.127321
3,3,-26.102104,1.336442,2.554289,0.218241
4,4,-30.241123,8.901708,3.240603,0.289511
5,5,-32.380389,11.997093,4.003917,0.197782
6,6,-32.414893,11.977719,4.887326,0.267062
7,7,-36.491183,13.143462,5.922597,0.807427
8,8,-37.821600,14.774109,6.174757,0.773586
9,9,-37.976847,14.751085,7.045220,2.190604


In [50]:
target = os.path.join(LOG_DIR, 'E2')
keywards = ["TP", 'Sinusoidal', 'EI']
filename_list = search_log_files(target, keywards)
print(filename_list)
filepath_list = [os.path.join(target, filename) for filename in filename_list]

dataframes = []
for filepath in filepath_list:
    parser = LogParser(filepath)
    parser.parse_log_file()
    df = parser.create_combined_dataframe()
    dataframes.append(df)

result_df = process_dataframes(dataframes)
result_df

['2024-09-21_22-18-31_vanilla_bo_SinusoidalSynthetic_TP_EI_seed0.log', '2024-09-21_22-22-01_vanilla_bo_SinusoidalSynthetic_TP_EI_seed1.log', '2024-09-21_22-22-01_vanilla_bo_SinusoidalSynthetic_TP_EI_seed2.log', '2024-09-21_22-22-01_vanilla_bo_SinusoidalSynthetic_TP_EI_seed3.log', '2024-09-21_22-22-01_vanilla_bo_SinusoidalSynthetic_TP_EI_seed4.log']
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 50
Maximum number of rows: 56


,Iteration,y_best_mean,y_best_std,beta_mean,beta_std
0,0,-39.265759,13.072124,2.173240,0.030504
1,1,-39.265759,14.245017,2.173240,0.033241
2,2,-39.265759,14.245017,2.963577,0.102731
3,3,-40.209085,12.965507,3.906513,0.236528
4,4,-40.885957,13.317178,4.894375,0.328746
5,5,-42.911349,14.919652,6.039124,0.702125
6,6,-43.473798,15.028472,7.509961,1.044225
7,7,-43.505324,15.057102,8.800836,0.913563
8,8,-43.550779,14.993953,9.436005,1.382060
9,9,-43.552102,14.995162,11.294768,1.362176


In [51]:
# Function to create and store result_df for each model
def create_result_df_dict(target, models, keywards):
    result_df_dict = {}
    
    # Loop through each model to process files and create result_df
    for model_name in models:
        # Update keywards with the current model name
        updated_keywards = [model_name] + keywards  # Replace the first item with model_name
        
        # Search for log files
        filename_list = search_log_files(target, updated_keywards)
        filepath_list = [os.path.join(target, filename) for filename in filename_list]
        
        # Initialize an empty list to hold dataframes
        dataframes = []
        
        # Parse each log file and create dataframes
        for filepath in filepath_list:
            parser = LogParser(filepath)
            parser.parse_log_file()
            df = parser.create_combined_dataframe()
            dataframes.append(df)
        
        # Process the dataframes and store the result in result_df_dict
        result_df = process_dataframes(dataframes)
        result_df_dict[model_name] = result_df  # Store the result in the dictionary with the model name as key
    
    return result_df_dict

Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 54
Maximum number of rows: 57
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 50
Maximum number of rows: 56
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid synt

# EI

In [60]:
target = os.path.join(LOG_DIR, 'E2')
models = ['GP', 'TP', 'AGT']
objective = 'SinusoidalSynthetic'
acquisition = 'EI'
keywards = [objective, acquisition]

result_df_dict = create_result_df_dict(target, models, keywards)

color_dict = {
    'GP': px.colors.qualitative.Plotly[0],
    'TP': px.colors.qualitative.Plotly[1],
    'AGT': px.colors.qualitative.Plotly[2],
}

fig = go.Figure()

for model_name, result_df in result_df_dict.items():
    add_history_plot_with_error_bars(fig, model_name, result_df)

# Customize the layout
fig.update_layout(
    title=f"Bayesian Optimization History on {objective} with {acquisition} Acquisition",
    xaxis_title="Iteration",
    yaxis_title="Best Function Value",
)

fig.show()

Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 54
Maximum number of rows: 57
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 50
Maximum number of rows: 56
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid synt

In [ ]:
target = os.path.join(LOG_DIR, 'E2')
models = ['GP', 'TP', 'AGT']
objective = 'SinusoidalSynthetic'
acquisition = 'EI'
keywards = [objective, acquisition]

result_df_dict = create_result_df_dict(target, models, keywards)

color_dict = {
    'GP': px.colors.qualitative.Plotly[0],
    'TP': px.colors.qualitative.Plotly[1],
    'AGT': px.colors.qualitative.Plotly[2],
}

fig = go.Figure()

for model_name, result_df in result_df_dict.items():
    add_history_plot_with_error_bars(fig, model_name, result_df)

# Customize the layout
fig.update_layout(
    title=f"Bayesian Optimization History on {objective} with {acquisition} Acquisition",
    xaxis_title="Iteration",
    yaxis_title="Best Function Value",
)

fig.show()

In [61]:
target = os.path.join(LOG_DIR, 'E2')
models = ['GP', 'TP', 'AGT']
objective = 'BraninHoo'
acquisition = 'EI'
keywards = [objective, acquisition]

result_df_dict = create_result_df_dict(target, models, keywards)

color_dict = {
    'GP': px.colors.qualitative.Plotly[0],
    'TP': px.colors.qualitative.Plotly[1],
    'AGT': px.colors.qualitative.Plotly[2],
}

fig = go.Figure()

for model_name, result_df in result_df_dict.items():
    add_history_plot_with_error_bars(fig, model_name, result_df)

# Customize the layout
fig.update_layout(
    title=f"Bayesian Optimization History on {objective} with {acquisition} Acquisition",
    xaxis_title="Iteration",
    yaxis_title="Best Function Value",
)

fig.show()

Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 63
Maximum number of rows: 71
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 53
Maximum number of rows: 63
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid synt

In [62]:
target = os.path.join(LOG_DIR, 'E2')
models = ['GP', 'TP', 'AGT']
objective = 'Hartmann6'
acquisition = 'EI'
keywards = [objective, acquisition]

result_df_dict = create_result_df_dict(target, models, keywards)

color_dict = {
    'GP': px.colors.qualitative.Plotly[0],
    'TP': px.colors.qualitative.Plotly[1],
    'AGT': px.colors.qualitative.Plotly[2],
}

fig = go.Figure()

for model_name, result_df in result_df_dict.items():
    add_history_plot_with_error_bars(fig, model_name, result_df)

# Customize the layout
fig.update_layout(
    title=f"Bayesian Optimization History on {objective} with {acquisition} Acquisition",
    xaxis_title="Iteration",
    yaxis_title="Best Function Value",
)

fig.show()

Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 94
Maximum number of rows: 104
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 104
Maximum number of rows: 104
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid s

# UCB

In [63]:
target = os.path.join(LOG_DIR, 'E2')
models = ['GP', 'TP', 'AGT']
objective = 'SinusoidalSynthetic'
acquisition = 'UCB'
keywards = [objective, acquisition]

result_df_dict = create_result_df_dict(target, models, keywards)

color_dict = {
    'GP': px.colors.qualitative.Plotly[0],
    'TP': px.colors.qualitative.Plotly[1],
    'AGT': px.colors.qualitative.Plotly[2],
}

fig = go.Figure()

for model_name, result_df in result_df_dict.items():
    add_history_plot_with_error_bars(fig, model_name, result_df)

# Customize the layout
fig.update_layout(
    title=f"Bayesian Optimization History on {objective} with {acquisition} Acquisition",
    xaxis_title="Iteration",
    yaxis_title="Best Function Value",
)

fig.show()

Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 52
Maximum number of rows: 56
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 45
Maximum number of rows: 50
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid synt

In [64]:
target = os.path.join(LOG_DIR, 'E2')
models = ['GP', 'TP', 'AGT']
objective = 'BraninHoo'
acquisition = 'UCB'
keywards = [objective, acquisition]

result_df_dict = create_result_df_dict(target, models, keywards)

color_dict = {
    'GP': px.colors.qualitative.Plotly[0],
    'TP': px.colors.qualitative.Plotly[1],
    'AGT': px.colors.qualitative.Plotly[2],
}

fig = go.Figure()

for model_name, result_df in result_df_dict.items():
    add_history_plot_with_error_bars(fig, model_name, result_df)

# Customize the layout
fig.update_layout(
    title=f"Bayesian Optimization History on {objective} with {acquisition} Acquisition",
    xaxis_title="Iteration",
    yaxis_title="Best Function Value",
)

fig.show()

Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 64
Maximum number of rows: 71
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 48
Maximum number of rows: 55
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid synt

In [65]:
target = os.path.join(LOG_DIR, 'E2')
models = ['GP', 'TP', 'AGT']
objective = 'Hartmann6'
acquisition = 'UCB'
keywards = [objective, acquisition]

result_df_dict = create_result_df_dict(target, models, keywards)

color_dict = {
    'GP': px.colors.qualitative.Plotly[0],
    'TP': px.colors.qualitative.Plotly[1],
    'AGT': px.colors.qualitative.Plotly[2],
}

fig = go.Figure()

for model_name, result_df in result_df_dict.items():
    add_history_plot_with_error_bars(fig, model_name, result_df)

# Customize the layout
fig.update_layout(
    title=f"Bayesian Optimization History on {objective} with {acquisition} Acquisition",
    xaxis_title="Iteration",
    yaxis_title="Best Function Value",
)

fig.show()

Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 92
Maximum number of rows: 104
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 82
Maximum number of rows: 104
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid sy

# POI

In [66]:
target = os.path.join(LOG_DIR, 'E2')
models = ['GP', 'TP', 'AGT']
objective = 'SinusoidalSynthetic'
acquisition = 'POI'
keywards = [objective, acquisition]

result_df_dict = create_result_df_dict(target, models, keywards)

color_dict = {
    'GP': px.colors.qualitative.Plotly[0],
    'TP': px.colors.qualitative.Plotly[1],
    'AGT': px.colors.qualitative.Plotly[2],
}

fig = go.Figure()

for model_name, result_df in result_df_dict.items():
    add_history_plot_with_error_bars(fig, model_name, result_df)

# Customize the layout
fig.update_layout(
    title=f"Bayesian Optimization History on {objective} with {acquisition} Acquisition",
    xaxis_title="Iteration",
    yaxis_title="Best Function Value",
)

fig.show()

Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 47
Maximum number of rows: 56
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 45
Maximum number of rows: 50
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid synt

In [67]:
target = os.path.join(LOG_DIR, 'E2')
models = ['GP', 'TP', 'AGT']
objective = 'BraninHoo'
acquisition = 'POI'
keywards = [objective, acquisition]

result_df_dict = create_result_df_dict(target, models, keywards)

color_dict = {
    'GP': px.colors.qualitative.Plotly[0],
    'TP': px.colors.qualitative.Plotly[1],
    'AGT': px.colors.qualitative.Plotly[2],
}

fig = go.Figure()

for model_name, result_df in result_df_dict.items():
    add_history_plot_with_error_bars(fig, model_name, result_df)

# Customize the layout
fig.update_layout(
    title=f"Bayesian Optimization History on {objective} with {acquisition} Acquisition",
    xaxis_title="Iteration",
    yaxis_title="Best Function Value",
)

fig.show()

Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 63
Maximum number of rows: 67
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 53
Maximum number of rows: 59
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid synt

In [68]:
target = os.path.join(LOG_DIR, 'E2')
models = ['GP', 'TP', 'AGT']
objective = 'Hartmann6'
acquisition = 'POI'
keywards = [objective, acquisition]

result_df_dict = create_result_df_dict(target, models, keywards)

color_dict = {
    'GP': px.colors.qualitative.Plotly[0],
    'TP': px.colors.qualitative.Plotly[1],
    'AGT': px.colors.qualitative.Plotly[2],
}

fig = go.Figure()

for model_name, result_df in result_df_dict.items():
    add_history_plot_with_error_bars(fig, model_name, result_df)

# Customize the layout
fig.update_layout(
    title=f"Bayesian Optimization History on {objective} with {acquisition} Acquisition",
    xaxis_title="Iteration",
    yaxis_title="Best Function Value",
)

fig.show()

Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 95
Maximum number of rows: 104
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Minimum number of rows: 104
Maximum number of rows: 104
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid syntax (<unknown>, line 1)
Failed to parse settings: invalid s